In [1]:
import pandas as pd
import numpy as np
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)    
    except FileNotFoundError as e:
        return False;


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Notebook for transforming & vectorizing all data except observations (to save RAM and because transformations are not the same). 
## Here we take all data for every patient and his every encounter and combine it to one vector. The final vector "columns" consist of all possible values for every data (e.g. a column can be a type of medication or a condition). Every patient then has 
## values in columns where appropriate. E.g. if a patient has been prescribed with "Penicilin" sometime (once) in his/hers lifetime, there will be a number 1 (penalized for time) in the "Penicilin" column of final vector.  

In [17]:
encounters = pd.read_csv('/content/drive/MyDrive/SIAP/data/encounters.csv')
print("encounters")
display(encounters.head(10))
conditions = pd.read_csv('/content/drive/MyDrive/SIAP/data/conditions.csv')
print("conditions")
display(conditions.head(10))
careplans = pd.read_csv('/content/drive/MyDrive/SIAP/data/careplans.csv')
print("careplans")
display(careplans.head(10))
devices = pd.read_csv('/content/drive/MyDrive/SIAP/data/devices.csv')
print("devices")
display(devices.head(10))
procedures = pd.read_csv('/content/drive/MyDrive/SIAP/data/procedures.csv')
print("procedures")
display(procedures.head(10))
medications = pd.read_csv('/content/drive/MyDrive/SIAP/data/medications.csv')
print("medications")
display(medications.head(10))

patients = pd.read_csv('/content/drive/MyDrive/SIAP/data/patients.csv')
print("patients")
display(patients.head(10))

encounters


,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,8d658e1b-68a4-bf3a-a51c-3a763fbfdab4,1990-03-19T18:32:29Z,1990-03-19T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424441002,Prenatal initial visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
1,0f1fb2ab-325e-3cc9-0009-7b3ad0ae3add,1991-06-17T18:32:29Z,1991-06-17T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,ambulatory,424441002,Prenatal initial visit,129.16,129.16,69.16,72892002.0,Normal pregnancy
2,e15cacc7-1277-b11a-218a-cd0c0c00043f,1994-06-16T19:19:29Z,1994-06-16T19:34:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185345009,Encounter for symptom,77.49,77.49,0.00,10509002.0,Acute bronchitis (disorder)
3,49b59550-cbb4-d496-f719-b3fd9d6b0bd4,1994-10-03T18:32:29Z,1994-10-03T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424441002,Prenatal initial visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
4,4d8e05e7-958c-6db7-88c5-cd11d8a11f9e,1995-01-02T18:32:29Z,1995-01-02T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424619006,Prenatal visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
5,9e5ae7a8-02a5-5d18-1958-e401c753746f,1995-01-30T18:32:29Z,1995-01-30T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424619006,Prenatal visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
6,e21d370e-d35f-4e48-5046-6a85c7363b2e,1995-02-27T18:32:29Z,1995-02-27T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424619006,Prenatal visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
7,34a7c63b-65cd-a145-9ec3-40ef95ad34ca,1995-03-27T18:32:29Z,1995-03-27T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424619006,Prenatal visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
8,db01521f-f231-4a57-6473-ed8b6e30eff9,1995-04-24T18:32:29Z,1995-04-24T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424619006,Prenatal visit,129.16,129.16,0.00,72892002.0,Normal pregnancy
9,582f5299-c6b5-6299-7a3c-bccae9cb3758,1995-05-08T18:32:29Z,1995-05-08T19:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,ef6ab57c-ed94-3dbe-9861-812d515918b3,fa15cdb0-a065-3015-9ee4-ecdd1c5cd595,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,emergency,183460006,Obstetric emergency hospital admission,129.16,129.16,0.00,72892002.0,Normal pregnancy


conditions


,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1990-03-19,NaN,0dddc561-6214-eac2-0ac8-ff88b4f89433,8d658e1b-68a4-bf3a-a51c-3a763fbfdab4,19169002,Miscarriage in first trimester
1,1991-06-17,NaN,0dddc561-6214-eac2-0ac8-ff88b4f89433,0f1fb2ab-325e-3cc9-0009-7b3ad0ae3add,79586000,Tubal pregnancy
2,1994-10-03,1995-05-08,0dddc561-6214-eac2-0ac8-ff88b4f89433,49b59550-cbb4-d496-f719-b3fd9d6b0bd4,72892002,Normal pregnancy
3,1994-10-03,1995-05-08,0dddc561-6214-eac2-0ac8-ff88b4f89433,49b59550-cbb4-d496-f719-b3fd9d6b0bd4,271737000,Anemia (disorder)
4,1995-09-25,1996-05-06,0dddc561-6214-eac2-0ac8-ff88b4f89433,c1f9a88a-51b3-81fa-3fca-73e0c884ad50,72892002,Normal pregnancy
5,1997-04-21,1997-11-24,0dddc561-6214-eac2-0ac8-ff88b4f89433,fa8b7f44-1b1d-d280-f65e-e8600ed8fb81,72892002,Normal pregnancy
6,1997-06-16,NaN,0dddc561-6214-eac2-0ac8-ff88b4f89433,38aafe98-2013-c890-6357-7d3a93757812,91302008,Sepsis (disorder)
7,1998-07-12,1998-07-19,0dddc561-6214-eac2-0ac8-ff88b4f89433,d37401e3-3720-7593-8169-4a4ec2a0f6cd,301011002,Escherichia coli urinary tract infection
8,1998-12-19,1998-12-27,0dddc561-6214-eac2-0ac8-ff88b4f89433,b17259cb-2c3e-111b-0e25-083409f59219,195662009,Acute viral pharyngitis (disorder)
9,1999-08-02,1999-08-30,0dddc561-6214-eac2-0ac8-ff88b4f89433,c60e2f9c-0749-b6bb-175c-b5b2d4041839,72892002,Normal pregnancy


careplans


...

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,1995-01-02T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,4d8e05e7-958c-6db7-88c5-cd11d8a11f9e,274804006,Evaluation of uterine fundal height,4370.10,72892002.0,Normal pregnancy
1,1995-01-02T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,4d8e05e7-958c-6db7-88c5-cd11d8a11f9e,225158009,Auscultation of the fetal heart,5372.77,72892002.0,Normal pregnancy
2,1995-01-30T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,9e5ae7a8-02a5-5d18-1958-e401c753746f,274804006,Evaluation of uterine fundal height,9788.47,72892002.0,Normal pregnancy
3,1995-01-30T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,9e5ae7a8-02a5-5d18-1958-e401c753746f,225158009,Auscultation of the fetal heart,7339.24,72892002.0,Normal pregnancy
4,1995-02-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,e21d370e-d35f-4e48-5046-6a85c7363b2e,104091002,Hemoglobin / Hematocrit / Platelet count,1728.62,72892002.0,Normal pregnancy
5,1995-02-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,e21d370e-d35f-4e48-5046-6a85c7363b2e,399014008,Vaccination for diphtheria pertussis and tet...,4355.16,72892002.0,Normal pregnancy
6,1995-02-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,e21d370e-d35f-4e48-5046-6a85c7363b2e,268556000,Urine screening for glucose,3570.03,72892002.0,Normal pregnancy
7,1995-02-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,e21d370e-d35f-4e48-5046-6a85c7363b2e,274804006,Evaluation of uterine fundal height,6008.99,72892002.0,Normal pregnancy
8,1995-02-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,e21d370e-d35f-4e48-5046-6a85c7363b2e,225158009,Auscultation of the fetal heart,8608.02,72892002.0,Normal pregnancy
9,1995-03-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,34a7c63b-65cd-a145-9ec3-40ef95ad34ca,274804006,Evaluation of uterine fundal height,3138.51,72892002.0,Normal pregnancy


medications


,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,1997-06-16T18:32:29Z,NaN,0dddc561-6214-eac2-0ac8-ff88b4f89433,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,38aafe98-2013-c890-6357-7d3a93757812,1659131,piperacillin 2000 MG / tazobactam 250 MG Injec...,263.49,0.0,213,56123.37,91302008.0,Sepsis (disorder)
1,1997-06-16T18:32:29Z,NaN,0dddc561-6214-eac2-0ac8-ff88b4f89433,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,38aafe98-2013-c890-6357-7d3a93757812,1807510,150 ML vancomycin 5 MG/ML Injection,263.49,0.0,213,56123.37,91302008.0,Sepsis (disorder)
2,1998-07-12T18:32:29Z,1998-07-19T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,d37401e3-3720-7593-8169-4a4ec2a0f6cd,311989,Nitrofurantoin 5 MG/ML Oral Suspension,1243.59,0.0,1,1243.59,301011002.0,Escherichia coli urinary tract infection
3,1998-07-12T18:32:29Z,1998-07-19T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,d37401e3-3720-7593-8169-4a4ec2a0f6cd,1094107,Phenazopyridine hydrochloride 100 MG Oral Tablet,23.53,0.0,1,23.53,301011002.0,Escherichia coli urinary tract infection
4,2002-09-26T18:32:29Z,2002-12-02T19:38:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1b8cb323-8389-99c1-22c7-a26dd7a81c96,748962,Camila 28 Day Pack,30.91,0.0,2,61.82,NaN,NaN
5,2003-12-13T18:43:29Z,2004-01-03T18:43:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,9c0e5ca9-78f9-4e7c-af32-6db8b0220066,849574,Naproxen sodium 220 MG Oral Tablet,19.21,0.0,1,19.21,NaN,NaN
6,2004-09-30T18:32:29Z,2005-09-25T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,929500a4-8cb4-0912-b395-58a3409b15f6,757594,Jolivette 28 Day Pack,39.37,0.0,12,472.44,NaN,NaN
7,2005-09-25T18:32:29Z,2006-09-20T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,b1531758-72d4-46f8-4df8-5b0f65df65dc,1359133,Estrostep Fe 28 Day Pack,44.47,0.0,12,533.64,NaN,NaN
8,2007-04-02T18:32:29Z,2008-03-27T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,c6161f2f-d583-62b6-a042-d9fcb945c854,831533,Errin 28 Day Pack,26.60,0.0,12,319.20,NaN,NaN
9,2008-03-27T18:32:29Z,2009-03-22T18:32:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,daca2684-15bd-85b6-defd-68d4fff6645c,757594,Jolivette 28 Day Pack,24.29,0.0,12,291.48,NaN,NaN


patients


,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,0dddc561-6214-eac2-0ac8-ff88b4f89433,1965-06-14,2014-12-08,999-76-5590,S99990367,X41914550X,Mrs.,Man114,Koepp521,NaN,Stroman228,M,white,nonhispanic,F,West Yarmouth Massachusetts US,351 Kirlin Light Suite 32,Marion,Massachusetts,Plymouth County,NaN,41.747323,-70.778742,61869.71,5022.83
1,34675498-b9c4-ea4a-47f1-1fd6ae929077,2001-01-01,2001-11-05,999-10-2345,NaN,NaN,NaN,Desire394,Marks830,NaN,NaN,NaN,white,nonhispanic,F,Malden Massachusetts US,970 Paucek Meadow Suite 36,Mashpee,Massachusetts,Barnstable County,NaN,41.612421,-70.463251,19180.00,294.96
2,6772742f-60b3-7826-dd06-bc01d045fd1c,2008-09-22,2015-05-04,999-30-4594,NaN,NaN,NaN,Hazel720,O'Keefe54,NaN,NaN,NaN,white,nonhispanic,F,East Bridgewater Massachusetts US,752 Cruickshank Tunnel,Haverhill,Massachusetts,Essex County,1830.0,42.752183,-71.142620,143478.30,2084.88
3,78327fb3-1419-66a0-b2d0-293e320940b4,1983-08-23,2014-07-22,999-92-6732,S99920583,X49372291X,Mr.,Cecil300,Grant908,NaN,NaN,M,white,nonhispanic,M,Westford Massachusetts US,485 Rolfson Path Suite 67,Falmouth,Massachusetts,Barnstable County,2540.0,41.622738,-70.673676,751846.41,6476.71
4,792dbb47-2b8d-78ad-9c7f-e50cf8332a4f,1968-09-28,2015-09-19,999-84-3602,S99976283,X691652X,Mr.,Clement78,Schumm995,NaN,NaN,M,white,nonhispanic,M,Springfield Massachusetts US,804 Lueilwitz Track Apt 4,Franklin,Massachusetts,Norfolk County,2038.0,42.049143,-71.426690,1075343.24,2795.44
5,1c138392-b8de-a786-06ed-d9236c9e9975,1971-08-03,1987-09-10,999-51-9046,S99958223,NaN,NaN,Asia282,Bailey598,NaN,NaN,NaN,white,nonhispanic,F,Boston Massachusetts US,900 Kuhlman Harbor Unit 85,Freetown,Massachusetts,Bristol County,NaN,41.808737,-71.004332,412612.89,3910.34
6,251358db-d8ee-29ed-e0a9-c546f5525b53,1993-11-27,1994-08-06,999-18-7063,NaN,NaN,NaN,Rodrigo242,Roob72,NaN,NaN,NaN,white,nonhispanic,M,Lancaster Massachusetts US,159 Grady Grove,Natick,Massachusetts,Middlesex County,NaN,42.329310,-71.326705,15700.00,245.80
7,dd139f1c-253a-7a5a-1f28-675cb1ec96ec,2016-05-04,2016-11-09,999-66-5644,NaN,NaN,NaN,Keren761,Jast432,NaN,NaN,NaN,asian,hispanic,F,Lawrence Massachusetts US,422 Macejkovic Crossing Unit 51,Ashby,Massachusetts,Middlesex County,NaN,42.694981,-71.853383,14000.00,516.64
8,ab4a30d1-722e-fb69-313c-890f13b246da,2015-11-29,2018-09-23,999-65-5998,NaN,NaN,NaN,Kelvin159,Hermiston71,NaN,NaN,NaN,white,nonhispanic,M,Westfield Massachusetts US,107 Daniel Heights Suite 83,Pittsfield,Massachusetts,Berkshire County,NaN,42.484290,-73.252471,65857.00,1587.40
9,a0ffce73-803e-d468-258f-9f24aac8f8ec,1981-06-09,2019-11-05,999-55-9085,S99919506,X47038261X,Mr.,Julio255,Kuvalis369,NaN,NaN,M,white,nonhispanic,M,Holliston Massachusetts US,438 DuBuque Trailer Apt 31,Kingston,Massachusetts,Plymouth County,NaN,41.940895,-70.718965,933804.69,9108.05


In [18]:
## FILTERING

# conditions = conditions[~conditions['DESCRIPTION'].isin(['Viral sinusitis (disorder)'])]
# conditions.shape
encounters = encounters[~encounters['REASONDESCRIPTION'].isin(['Bullet wound', 'Drug overdose', 'Suicide - firearms', 'Suicide - suffocation'])]

In [19]:
# patients_temp = patients[['Id', 'BIRTHDATE', 'DEATHDATE']]
# patients_temp['YEARS'] = (patients_temp['DEATHDATE'].apply(pd.to_datetime) - patients_temp['BIRTHDATE'].apply(pd.to_datetime)).dt.days / 365
patients_death = patients[['Id', 'DEATHDATE']]
patients_death.head()


,Id,DEATHDATE
0,0dddc561-6214-eac2-0ac8-ff88b4f89433,2014-12-08
1,34675498-b9c4-ea4a-47f1-1fd6ae929077,2001-11-05
2,6772742f-60b3-7826-dd06-bc01d045fd1c,2015-05-04
3,78327fb3-1419-66a0-b2d0-293e320940b4,2014-07-22
4,792dbb47-2b8d-78ad-9c7f-e50cf8332a4f,2015-09-19


In [20]:
encounters['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(encounters.shape[0], dtype=float)
conditions['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(conditions.shape[0], dtype=float)
medications['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(medications.shape[0], dtype=float)
procedures['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(procedures.shape[0], dtype=float)
devices['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(devices.shape[0], dtype=float)
careplans['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(careplans.shape[0], dtype=float)


In [21]:
encounters = encounters.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')
conditions = conditions.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')
careplans = careplans.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')
devices = devices.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')
procedures = procedures.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')
medications = medications.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')


In [22]:
encounters['DEATHDATE'] = pd.to_datetime(encounters['DEATHDATE']).dt.date
encounters['START'] = pd.to_datetime(encounters['START']).dt.date

conditions['DEATHDATE'] = pd.to_datetime(conditions['DEATHDATE']).dt.date
conditions['START'] = pd.to_datetime(conditions['START']).dt.date

careplans['DEATHDATE'] = pd.to_datetime(careplans['DEATHDATE']).dt.date
careplans['START'] = pd.to_datetime(careplans['START']).dt.date

devices['DEATHDATE'] = pd.to_datetime(devices['DEATHDATE']).dt.date
devices['START'] = pd.to_datetime(devices['START']).dt.date

procedures['DEATHDATE'] = pd.to_datetime(procedures['DEATHDATE']).dt.date
procedures['DATE'] = pd.to_datetime(procedures['DATE']).dt.date

medications['DEATHDATE'] = pd.to_datetime(medications['DEATHDATE']).dt.date
medications['START'] = pd.to_datetime(medications['START']).dt.date

In [23]:
encounters['yfd'] = (encounters['DEATHDATE'] - encounters['START']).dt.days / 365
conditions['yfd'] = (conditions['DEATHDATE'] - conditions['START']).dt.days / 365
careplans['yfd'] = (careplans['DEATHDATE'] - careplans['START']).dt.days / 365
devices['yfd'] = (devices['DEATHDATE'] - devices['START']).dt.days / 365
procedures['yfd'] = (procedures['DEATHDATE'] - procedures['DATE']).dt.days / 365
medications['yfd'] = (medications['DEATHDATE'] - medications['START']).dt.days / 365


In [24]:
# Default vrednosti za vremenski znacaj
morethan30 = 0.2
morethan20 = 0.4
morethan5 = 0.6
morethan1 = 0.9
lessthan1 = 1

ignore_less_than_0_3 = 0.0

cnt = 0

# Funkcija za dodeljivanje vrednosti koloni YEARS_FROM_DEATH_WEIGHT
# 
def transform_row(r):
    yfd = r['yfd']
    if yfd >= 30:
      r['YEARS_FROM_DEATH_WEIGHT'] = morethan30
    elif yfd < 30 and yfd >= 20:
      r['YEARS_FROM_DEATH_WEIGHT'] = morethan20
    elif yfd < 20 and yfd >= 5:
      r['YEARS_FROM_DEATH_WEIGHT'] = morethan5
    elif yfd < 5 and yfd >= 1:
      r['YEARS_FROM_DEATH_WEIGHT'] = morethan1
    elif yfd < 1 and yfd >= 0.6:
      r['YEARS_FROM_DEATH_WEIGHT'] = lessthan1
    else:
      r['YEARS_FROM_DEATH_WEIGHT'] = ignore_less_than_0_3
    return r

encounters_with_time_weight = encounters.apply(transform_row, axis=1)
conditions_with_time_weight = conditions.apply(transform_row, axis=1)
careplans_with_time_weight = careplans.apply(transform_row, axis=1)
devices_with_time_weight = devices.apply(transform_row, axis=1)
procedures_with_time_weight = procedures.apply(transform_row, axis=1)
medications_with_time_weight = medications.apply(transform_row, axis=1)

In [25]:
encounters_pivoted = pd.pivot_table(encounters_with_time_weight, index=['PATIENT'], columns=['REASONDESCRIPTION'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)
conditions_pivoted = pd.pivot_table(conditions_with_time_weight, index=['PATIENT'], columns=['DESCRIPTION'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)
careplans_pivoted = pd.pivot_table(careplans_with_time_weight, index=['PATIENT'], columns=['DESCRIPTION'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)
devices_pivoted = pd.pivot_table(devices_with_time_weight, index=['PATIENT'], columns=['DESCRIPTION'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)
procedures_pivoted = pd.pivot_table(procedures_with_time_weight, index=['PATIENT'], columns=['DESCRIPTION'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)
medications_pivoted = pd.pivot_table(medications_with_time_weight, index=['PATIENT'], columns=['DESCRIPTION'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)


In [ ]:


conditions_pivoted = pd.pivot(conditions[['ENCOUNTER', 'PATIENT', 'DESCRIPTION', 'START']], index=['ENCOUNTER', 'PATIENT'], columns=['DESCRIPTION'])
conditions_pivoted.fillna(0)

START  ...                        
DESCRIPTION                                                               Acquired coagulation disorder (disorder)  ... Whiplash injury to neck
ENCOUNTER                            PATIENT                                                                        ...                        
00008289-83ee-2769-3de1-6620e256a601 14dc8ba3-20b0-e131-46b8-9ebcb92f95f8                                        0  ...                       0
0000e03b-2c8b-14d1-aa01-c1e56416f6df 65ab6d59-20e2-04fb-f6ea-26374ed3b0fc                                        0  ...                       0
00010493-3a4b-51ed-9690-36f70ca1a70a b86f18c4-79ea-4122-5cdd-c5dae18b89b7                                        0  ...                       0
00014e5b-1e4d-2d3c-f2b0-6d9b3829b184 0ada12e4-bf8b-7a8a-48c1-05922d2f1ada                                        0  ...                       0
0001ca8d-0099-0840-543b-ffb97015b0bf 5a3ccfa3-475c-ec49-ac9a-642881c43536                                        0  ...                       0
...                                                                                                            ...  ...                     ...
fffcfe1b-241d-a670-a3b0-4afdb5d12a46 29bad601-9a82-3918-a0c7-b7a82aa0223a                                        0  ...                       0
fffda226-4400-5d04-d106-36de14b33d80 b077e680-5e96-d82a-1a51-275a98e2d386                                        0  ...                       0
fffef864-ea9d-0a40-4b89-78b40e5be0dd 17c4f199-f641-83f8-c6b0-3bb9846445e7                                        0  ...                       0
ffffb29e-16d8-0074-f656-cf30d41c7eb6 9bb60dff-289a-4eb6-b964-2a65e8f74594                                        0  ...                       0
ffffcef3-5573-0f3a-ae61-4cc88f6a350a f7a3f1c3-7663-88aa-bb40-e5e9cee24c98                                        0  ...                       0

[178101 rows x 222 columns]

In [ ]:
conditions_pivoted.head(20)

In [26]:
normalized_encounters_pivoted=(encounters_pivoted-encounters_pivoted.min())/(encounters_pivoted.max()-encounters_pivoted.min())
normalized_conditions_pivoted=(conditions_pivoted-conditions_pivoted.min())/(conditions_pivoted.max()-conditions_pivoted.min())
normalized_careplans_pivoted=(careplans_pivoted-careplans_pivoted.min())/(careplans_pivoted.max()-careplans_pivoted.min())
normalized_devices_pivoted=(devices_pivoted-devices_pivoted.min())/(devices_pivoted.max()-devices_pivoted.min())
normalized_procedures_pivoted=(procedures_pivoted-procedures_pivoted.min())/(procedures_pivoted.max()-procedures_pivoted.min())
normalized_medications_pivoted=(medications_pivoted-medications_pivoted.min())/(medications_pivoted.max()-medications_pivoted.min())

In [27]:
normalized_encounters_pivoted.columns = normalized_encounters_pivoted.columns.droplevel(0)
normalized_conditions_pivoted.columns = normalized_conditions_pivoted.columns.droplevel(0)
normalized_careplans_pivoted.columns = normalized_careplans_pivoted.columns.droplevel(0)
normalized_devices_pivoted.columns = normalized_devices_pivoted.columns.droplevel(0)
normalized_procedures_pivoted.columns = normalized_procedures_pivoted.columns.droplevel(0)
normalized_medications_pivoted.columns = normalized_medications_pivoted.columns.droplevel(0)


In [28]:
patients_join = patients[['Id']]

join = normalized_encounters_pivoted.join(patients_join.set_index('Id'), on='PATIENT', how='right', lsuffix='_E')
join = normalized_conditions_pivoted.join(join.set_index('PATIENT'), on='PATIENT', how='right', lsuffix='_CO')
join = normalized_careplans_pivoted.join(join.set_index('PATIENT'), on='PATIENT', how='right', lsuffix='_CP')
join = normalized_devices_pivoted.join(join.set_index('PATIENT'), on='PATIENT', how='right', lsuffix='_D')
join = normalized_procedures_pivoted.join(join.set_index('PATIENT'), on='PATIENT', how='right', lsuffix='_P')
join = normalized_medications_pivoted.join(join.set_index('PATIENT'), on='PATIENT', how='right', lsuffix='_M')


In [29]:
join.shape

(20999, 830)

In [30]:
join.to_csv('/content/drive/MyDrive/SIAP/data/prepared/without_observations.csv', encoding='utf-8', index=False)


In [ ]:
combined_observations = pd.read_csv('/content/drive/MyDrive/SIAP/data/prepared/observations_combined_with_buckets.csv')

combined_observations.head()

,PATIENT,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_0,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_1,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_2,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_3,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_4,o_Albumin [Mass/volume] in Serum or Plasma_0,o_Albumin [Mass/volume] in Serum or Plasma_1,o_Albumin [Mass/volume] in Serum or Plasma_2,o_Albumin [Mass/volume] in Serum or Plasma_3,o_Albumin [Mass/volume] in Serum or Plasma_4,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_0,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_1,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_2,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_3,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_4,o_American house dust mite IgE Ab in Serum_0,o_American house dust mite IgE Ab in Serum_1,o_American house dust mite IgE Ab in Serum_2,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_0,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_1,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_2,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_3,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_4,o_Basophils [#/volume] in Blood by Automated count_0,o_Basophils [#/volume] in Blood by Automated count_1,o_Basophils [#/volume] in Blood by Automated count_3,o_Basophils/100 leukocytes in Blood by Automated count_0,o_Basophils/100 leukocytes in Blood by Automated count_1,o_Basophils/100 leukocytes in Blood by Automated count_2,o_Basophils/100 leukocytes in Blood by Automated count_3,o_Basophils/100 leukocytes in Blood by Automated count_4,o_Bicarbonate [Moles/volume] in Arterial blood_0,o_Bicarbonate [Moles/volume] in Arterial blood_1,o_Bicarbonate [Moles/volume] in Arterial blood_2,o_Bicarbonate [Moles/volume] in Arterial blood_3,o_Bicarbonate [Moles/volume] in Arterial blood_4,o_Bilirubin.total [Mass/volume] in Serum or Plasma_0,o_Bilirubin.total [Mass/volume] in Serum or Plasma_1,o_Bilirubin.total [Mass/volume] in Serum or Plasma_2,...,o_Weight difference [Mass difference] --pre dialysis - post dialysis_0,o_Weight difference [Mass difference] --pre dialysis - post dialysis_1,o_Weight difference [Mass difference] --pre dialysis - post dialysis_2,o_Weight difference [Mass difference] --pre dialysis - post dialysis_3,o_Weight difference [Mass difference] --pre dialysis - post dialysis_4,o_Weight-for-length Per age and sex_0,o_Weight-for-length Per age and sex_1,o_Weight-for-length Per age and sex_2,o_Weight-for-length Per age and sex_3,o_Weight-for-length Per age and sex_4,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_0,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_1,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_2,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_3,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_4,o_What number best describes how during the past week pain has interfered with your general activity?_0,o_What number best describes how during the past week pain has interfered with your general activity?_1,o_What number best describes how during the past week pain has interfered with your general activity?_2,o_What number best describes how during the past week pain has interfered with your general activity?_3,o_What number best describes how during the past week pain has interfered with your general activity?_4,o_What number best describes your pain on average in the pa

In [ ]:
complete = join.join(combined_observations.set_index('PATIENT'), on='PATIENT', how='left')


In [ ]:
print(complete.shape)
print(len(complete['PATIENT'].unique()))



(20999, 1425)
20999


In [ ]:
final = complete.groupby(complete['PATIENT']).aggregate('sum')
final = final.reset_index()

In [ ]:
death_encounters = encounters[encounters['DESCRIPTION'] == 'Death Certification']
death_encounters.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION,YEARS_FROM_DEATH_WEIGHT,DEATHDATE,yfd
120,9aa9bd89-47a4-06fc-3f84-b7c94144c448,2014-12-15,2014-12-15T18:47:29Z,0dddc561-6214-eac2-0ac8-ff88b4f89433,e9e2998f-97d3-3f88-8ee8-82660c5ddfce,fcfa64a6-93a0-3a32-b95b-23fb061a66a6,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,308646001,Death Certification,129.16,129.16,0.0,9855000.0,Natural death with unknown cause,0.0,2014-12-08,-0.019178
127,70ea243e-4f8c-9765-f9ba-7f6ecff4bd23,2001-11-12,2001-11-12T07:01:57Z,34675498-b9c4-ea4a-47f1-1fd6ae929077,deb5737f-1734-3246-b0ab-e2b9534542ec,589cbd68-4ec5-3264-b436-0573fd4a033e,d47b3510-2895-3b70-9897-342d681c769d,wellness,308646001,Death Certification,129.16,129.16,0.0,95281009.0,Sudden Cardiac Death,0.0,2001-11-05,-0.019178
156,d4ee6c18-b51d-0fb0-23c5-f1ba479af3f3,2015-05-11,2015-05-11T08:16:15Z,6772742f-60b3-7826-dd06-bc01d045fd1c,cb1e1fea-a91a-3ce9-a3bd-24d0b77c334c,c206b077-c294-3da7-976a-883a86744ad5,d47b3510-2895-3b70-9897-342d681c769d,wellness,308646001,Death Certification,129.16,129.16,0.0,22298006.0,Myocardial Infarction,0.0,2015-05-04,-0.019178
247,d8da75db-ce6f-1cf4-3104-2d520c5704cc,2014-07-29,2014-07-29T12:04:34Z,78327fb3-1419-66a0-b2d0-293e320940b4,bee99f4b-dbff-334b-ba02-f90f8e199aca,5441a12a-3b77-3d41-995f-2bfd31da3827,4d71f845-a6a9-3c39-b242-14d25ef86a8d,wellness,308646001,Death Certification,129.16,129.16,0.0,9855000.0,Natural death with unknown cause,0.0,2014-07-22,-0.019178
268,79ffa80f-8112-ca02-e04e-1b91795ded41,2015-09-26,2015-09-26T09:22:26Z,792dbb47-2b8d-78ad-9c7f-e50cf8332a4f,7582039c-4265-3c65-afbe-38fe8089afe8,5e083fe4-f084-3009-9b02-49df1aab2e75,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,wellness,308646001,Death Certification,129.16,129.16,0.0,9855000.0,Natural death with unknown cause,0.0,2015-09-19,-0.019178


In [ ]:
patients_death_reason = death_encounters.join(patients_death.set_index('Id'), on='PATIENT', how='right', lsuffix="_L")
patients_death_reason = patients_death_reason[['PATIENT', 'REASONDESCRIPTION']]


In [ ]:
patients_death_reason.head()

,PATIENT,REASONDESCRIPTION
120.0,0dddc561-6214-eac2-0ac8-ff88b4f89433,Natural death with unknown cause
127.0,34675498-b9c4-ea4a-47f1-1fd6ae929077,Sudden Cardiac Death
156.0,6772742f-60b3-7826-dd06-bc01d045fd1c,Myocardial Infarction
247.0,78327fb3-1419-66a0-b2d0-293e320940b4,Natural death with unknown cause
268.0,792dbb47-2b8d-78ad-9c7f-e50cf8332a4f,Natural death with unknown cause


In [ ]:
to_save = final.join(patients_death_reason.set_index('PATIENT'), on='PATIENT', how='left')

In [ ]:
to_save.head()


,PATIENT,0.25 ML Leuprolide Acetate 30 MG/ML Prefilled Syringe,0.3 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe,0.4 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe,0.67 ML anakinra 149 MG/ML Prefilled Syringe,1 ML DOCEtaxel 20 MG/ML Injection,1 ML Enoxaparin sodium 150 MG/ML Prefilled Syringe,1 ML Epinephrine 1 MG/ML Injection,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen],1 ML Morphine Sulfate 5 MG/ML Injection,1 ML Vasopressin (USP) 20 UNT/ML Injection,1 ML denosumab 60 MG/ML Prefilled Syringe,1 ML heparin sodium porcine 5000 UNT/ML Injection,1 ML medroxyPROGESTERone acetate 150 MG/ML Injection,1 ML medroxyprogesterone acetate 150 MG/ML Injection,10 ML Alfentanil 0.5 MG/ML Injection,10 ML Doxorubicin Hydrochloride 2 MG/ML Injection,10 ML Fentanyl 0.05 MG/ML Injection,10 ML Fluorouracil 50 MG/ML Injection,10 ML Furosemide 10 MG/ML Injection,10 ML Pamidronate Disodium 3 MG/ML Injection,10 ML oxaliplatin 5 MG/ML Injection,100 ML Epirubicin Hydrochloride 2 MG/ML Injection,100 ML Propofol 10 MG/ML Injection,100 ML zoledronic acid 0.04 MG/ML Injection,12 HR Cefaclor 500 MG Extended Release Oral Tablet,12 HR Hydrocodone Bitartrate 10 MG Extended Release Oral Capsule,120 ACTUAT Fluticasone propionate 0.044 MG/ACTUAT Metered Dose Inhaler,150 ML vancomycin 5 MG/ML Injection,168 HR Ethinyl Estradiol 0.00146 MG/HR / norelgestromin 0.00625 MG/HR Transdermal System,2 ML Ondansetron 2 MG/ML Injection,20 ML Ciprofloxacin 10 MG/ML Injection,20 ML tocilizumab 20 MG/ML Injection,24 HR Donepezil hydrochloride 10 MG / Memantine hydrochloride 28 MG Extended Release Oral Capsule,24 HR Metformin hydrochloride 500 MG Extended Release Oral Tablet,24hr nicotine transdermal patch,3 ML Amiodarone hydrocholoride 50 MG/ML Prefilled Syringe,3 ML liraglutide 6 MG/ML Pen Injector,4 ML Norepinephrine 1 MG/ML Injection,4 ML norepinephrine 1 MG/ML Injection,...,o_Weight difference [Mass difference] --pre dialysis - post dialysis_1,o_Weight difference [Mass difference] --pre dialysis - post dialysis_2,o_Weight difference [Mass difference] --pre dialysis - post dialysis_3,o_Weight difference [Mass difference] --pre dialysis - post dialysis_4,o_Weight-for-length Per age and sex_0,o_Weight-for-length Per age and sex_1,o_Weight-for-length Per age and sex_2,o_Weight-for-length Per age and sex_3,o_Weight-for-length Per age and sex_4,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_0,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_1,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_2,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_3,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_4,o_What number best describes how during the past week pain has interfered with your general activity?_0,o_What number best describes how during the past week pain has interfered with your general activity?_1,o_What number best describes how during the past week pain has interfered with your general activity?_2,o_What number best describes how during the past week pain has interfered with your general activity?_3,o_What number best describes how during the past week pain has interfered with your general activity?_4,o_What number best describes your pain on average in the past week?_0,o_What number best describes your pain on average in the past week?_1,o_What number best describes your pain on average in the past week?_2,o_What number best describes your pain on average in the past week?_3,o_What number best describes your pain on average in the past week?_4,o_Wheat IgE Ab in Serum_0,o_Wheat IgE Ab in Serum_1,o_Wheat IgE Ab in Serum_2,o_White oak IgE Ab in Serum_0,o_White oak IgE Ab in Serum_1,o_White oak IgE Ab in Serum_2,o_pH of Arterial blood_0,o_pH of Arterial blood_1,o_pH of Arterial blood_3,o_pH of Arterial blood

In [ ]:
to_save.to_csv('/content/drive/MyDrive/SIAP/data/prepared/final_vectors_with_buckets.csv', encoding='utf-8', index=False)


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/SIAP/data/prepared/final_vectors_with_buckets.csv')
test.head()


,PATIENT,0.25 ML Leuprolide Acetate 30 MG/ML Prefilled Syringe,0.3 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe,0.4 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe,0.67 ML anakinra 149 MG/ML Prefilled Syringe,1 ML DOCEtaxel 20 MG/ML Injection,1 ML Enoxaparin sodium 150 MG/ML Prefilled Syringe,1 ML Epinephrine 1 MG/ML Injection,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen],1 ML Morphine Sulfate 5 MG/ML Injection,1 ML Vasopressin (USP) 20 UNT/ML Injection,1 ML denosumab 60 MG/ML Prefilled Syringe,1 ML heparin sodium porcine 5000 UNT/ML Injection,1 ML medroxyPROGESTERone acetate 150 MG/ML Injection,1 ML medroxyprogesterone acetate 150 MG/ML Injection,10 ML Alfentanil 0.5 MG/ML Injection,10 ML Doxorubicin Hydrochloride 2 MG/ML Injection,10 ML Fentanyl 0.05 MG/ML Injection,10 ML Fluorouracil 50 MG/ML Injection,10 ML Furosemide 10 MG/ML Injection,10 ML Pamidronate Disodium 3 MG/ML Injection,10 ML oxaliplatin 5 MG/ML Injection,100 ML Epirubicin Hydrochloride 2 MG/ML Injection,100 ML Propofol 10 MG/ML Injection,100 ML zoledronic acid 0.04 MG/ML Injection,12 HR Cefaclor 500 MG Extended Release Oral Tablet,12 HR Hydrocodone Bitartrate 10 MG Extended Release Oral Capsule,120 ACTUAT Fluticasone propionate 0.044 MG/ACTUAT Metered Dose Inhaler,150 ML vancomycin 5 MG/ML Injection,168 HR Ethinyl Estradiol 0.00146 MG/HR / norelgestromin 0.00625 MG/HR Transdermal System,2 ML Ondansetron 2 MG/ML Injection,20 ML Ciprofloxacin 10 MG/ML Injection,20 ML tocilizumab 20 MG/ML Injection,24 HR Donepezil hydrochloride 10 MG / Memantine hydrochloride 28 MG Extended Release Oral Capsule,24 HR Metformin hydrochloride 500 MG Extended Release Oral Tablet,24hr nicotine transdermal patch,3 ML Amiodarone hydrocholoride 50 MG/ML Prefilled Syringe,3 ML liraglutide 6 MG/ML Pen Injector,4 ML Norepinephrine 1 MG/ML Injection,4 ML norepinephrine 1 MG/ML Injection,...,o_Weight difference [Mass difference] --pre dialysis - post dialysis_1,o_Weight difference [Mass difference] --pre dialysis - post dialysis_2,o_Weight difference [Mass difference] --pre dialysis - post dialysis_3,o_Weight difference [Mass difference] --pre dialysis - post dialysis_4,o_Weight-for-length Per age and sex_0,o_Weight-for-length Per age and sex_1,o_Weight-for-length Per age and sex_2,o_Weight-for-length Per age and sex_3,o_Weight-for-length Per age and sex_4,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_0,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_1,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_2,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_3,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_4,o_What number best describes how during the past week pain has interfered with your general activity?_0,o_What number best describes how during the past week pain has interfered with your general activity?_1,o_What number best describes how during the past week pain has interfered with your general activity?_2,o_What number best describes how during the past week pain has interfered with your general activity?_3,o_What number best describes how during the past week pain has interfered with your general activity?_4,o_What number best describes your pain on average in the past week?_0,o_What number best describes your pain on average in the past week?_1,o_What number best describes your pain on average in the past week?_2,o_What number best describes your pain on average in the past week?_3,o_What number best describes your pain on average in the past week?_4,o_Wheat IgE Ab in Serum_0,o_Wheat IgE Ab in Serum_1,o_Wheat IgE Ab in Serum_2,o_White oak IgE Ab in Serum_0,o_White oak IgE Ab in Serum_1,o_White oak IgE Ab in Serum_2,o_pH of Arterial blood_0,o_pH of Arterial blood_1,o_pH of Arterial blood_3,o_pH of Arterial blood